# Daily CSV Combination & Standardization
*Jamie Prezioso, September 19, 2020*

This notebook is testing how to standardize daily CSV files from the COVID-19 dataset for D4BL.

**Completed** - *Goal of version 2 - Earlier runs didn’t include the run date, so that column will always be missing for those tables. You could extract the run date from the file name and insert it into the empty record. Accomplished in _v2*

**Completed** - *Goal of version 3 - Is there a way to concatenate the dataframes without including the row numbers (indices)? My preference is to have the index of the final table as the set of consecutive integers 1-n or 0-(n-1), without duplicate indices.*
Including the `index=True` argument fixes this:  `dfCombined.to_csv("combinedData{}.csv".format(date_object), index=True)`

**Completed** *Could you also sort the final table by run date and by location?* Sorted by date and location (alphabetically), and checked for duplicate locations.  *Note, when checking for duplicate locations, this prints a warning and exits the loop, the CSV merging does not complete.*

In [3]:
import os
import glob
import pandas as pd
import shutil
import numpy as np

## Test with a few CSV files

### Grab three different CSVs

In [4]:
# Test files 
f1 = '../csv/covid_disparities_output_2020-08-01.csv'
f2 = '../csv/covid_disparities_output_2020-08-19.csv' # Most recent
f3 = '../csv/covid_disparities_output_2020-07-01.csv'

In [5]:
df1 = pd.read_csv(f1)
c1 = list(df1.columns)
print('Filename: ' + f1)
df1.head()

Filename: ../csv/covid_disparities_output_2020-08-01.csv


,Location,Date Published,Total Cases,Total Deaths,Count Cases Black/AA,Count Deaths Black/AA,Pct Cases Black/AA,Pct Deaths Black/AA,Pct Includes Unknown Race,Pct Includes Hispanic Black,Count Cases Known Race,Count Deaths Known Race,Black/AA Population,Pct Black/AA Population,Status code
0,Illinois,2020-08-01,180476.0,7503.0,29784.0,2059.0,16.50,27.44,True,False,NaN,NaN,1824125.0,14.23,Success!
1,Louisiana,2020-07-29,116280.0,3835.0,41962.0,1884.0,44.38,50.36,False,True,94556.0,3741.0,1502916.0,32.23,Success!
2,Washington,2020-08-01,57541.0,1592.0,2134.0,51.0,5.49,3.29,False,False,38840.0,1551.0,269854.0,3.70,Success!
3,North Carolina,2020-08-01,123878.0,1964.0,21584.0,591.0,24.06,31.30,False,True,89713.0,1888.0,2179622.0,21.46,Success!
4,Wyoming,2020-08-01,2769.0,26.0,27.0,0.0,1.01,0.00,False,False,2669.0,25.0,5540.0,0.95,Success!


In [6]:
print('Filename: ' + f2)
df2 = pd.read_csv(f2)
c2 = list(df2.columns)
df2.head()

Filename: ../csv/covid_disparities_output_2020-08-19.csv


,Date Run,Date/Time Run,Location,Date Published,Total Cases,Total Deaths,Count Cases Black/AA,Count Deaths Black/AA,Pct Cases Black/AA,Pct Deaths Black/AA,Pct Includes Unknown Race,Pct Includes Hispanic Black,Count Cases Known Race,Count Deaths Known Race,Black/AA Population,Pct Black/AA Population,Status code
0,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ,Alabama,2020-08-19,106784.0,1876.0,28664.0,769.0,41.92,42.51,False,False,68376.0,1809.0,1293186.0,26.58,Success!
1,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ,Alaska,2020-08-19,2731.0,28.0,154.0,0.0,6.31,0.00,False,True,2441.0,28.0,24129.0,3.27,Success!
2,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ,Arizona,2020-08-19,195557.0,4634.0,4918.0,142.0,4.50,3.55,False,False,109208.0,3997.0,305259.0,4.39,Success!
3,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ,Arkansas,2020-08-19,54216.0,631.0,11909.0,163.0,24.59,25.83,False,True,48431.0,631.0,460970.0,15.41,Success!
4,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ,California,2020-08-17 00:00:00,632667.0,11342.0,17615.0,876.0,4.28,8.03,False,False,411365.0,10910.0,2267875.0,5.79,Success!


In [7]:
df3 = pd.read_csv(f3)
print('Filename: ' + f3)
c3 = list(df3.columns)
df3.head()

Filename: ../csv/covid_disparities_output_2020-07-01.csv


,Location,Date Published,Total Cases,Total Deaths,Count Cases Black/AA,Count Deaths Black/AA,Pct Cases Black/AA,Pct Deaths Black/AA,Pct Includes Unknown Race,Pct Includes Hispanic Black,Status code
0,Washington,2020-07-01,33435.0,1339.0,1366.0,43.0,6.00,3.00,False,False,Success!
1,North Carolina,2020-07-01,66513.0,1373.0,10818.0,436.0,16.26,31.76,False,True,Success!
2,NewYorkCity,2020-07-01,212412.0,18497.0,33061.0,5166.0,30.16,30.48,False,True,Success!
3,Wisconsin,2020-07-01,29199.0,786.0,5085.0,185.0,19.43,23.87,False,True,Success!
4,Wisconsin -- Milwaukee,2020-07-01,11553.0,355.0,3289.0,143.0,30.77,40.28,False,False,Success!


In [8]:
# Do any of the 3 data sets have the same col. headers?
print(set(c1) == set(c2))
print(set(c2) == set(c3))
print(set(c1) == set(c3))

False
False
False


In [9]:
print(len(c1))
print(len(c2))
print(len(c3))

15
17
11


In [10]:
c1

['Location',
 'Date Published',
 'Total Cases',
 'Total Deaths',
 'Count Cases Black/AA',
 'Count Deaths Black/AA',
 'Pct Cases Black/AA',
 'Pct Deaths Black/AA',
 'Pct Includes Unknown Race',
 'Pct Includes Hispanic Black',
 'Count Cases Known Race',
 'Count Deaths Known Race',
 'Black/AA Population',
 'Pct Black/AA Population',
 'Status code']

In [11]:
c2

['Date Run',
 'Date/Time Run',
 'Location',
 'Date Published',
 'Total Cases',
 'Total Deaths',
 'Count Cases Black/AA',
 'Count Deaths Black/AA',
 'Pct Cases Black/AA',
 'Pct Deaths Black/AA',
 'Pct Includes Unknown Race',
 'Pct Includes Hispanic Black',
 'Count Cases Known Race',
 'Count Deaths Known Race',
 'Black/AA Population',
 'Pct Black/AA Population',
 'Status code']

In [12]:
c3

['Location',
 'Date Published',
 'Total Cases',
 'Total Deaths',
 'Count Cases Black/AA',
 'Count Deaths Black/AA',
 'Pct Cases Black/AA',
 'Pct Deaths Black/AA',
 'Pct Includes Unknown Race',
 'Pct Includes Hispanic Black',
 'Status code']

### Combine those three CSVs

Using `pandas.DataFrame.merge` essentially takes the union of the column headers.  If a dataset does not have a column, but the set it is being merged with does, those column entries are set to NaN.  Alternatively, we could use `pandas.concat`.  Let's try both...

I think `pandas.concat` is the way to go for this since we're merging multiple dataframes into one dataframe (and then exporting as a CSV).  The merging option is a method of an individual data frame so this seems a bit clunky to do for multiple at once, however, once we have the full, up-to-date time series CSV, then we could use `merge` to perform a daily update.

In [13]:
# Downselect for easy inspection:
df1sm = df1[0:3]
df2sm = df2[0:4]
df3sm = df3[0:2]

In [14]:
outer_merged = df1sm.merge(df2sm, how="outer")
outer_merged.head()
print(outer_merged.shape)
outer_merged

(7, 17)


,Location,Date Published,Total Cases,Total Deaths,Count Cases Black/AA,Count Deaths Black/AA,Pct Cases Black/AA,Pct Deaths Black/AA,Pct Includes Unknown Race,Pct Includes Hispanic Black,Count Cases Known Race,Count Deaths Known Race,Black/AA Population,Pct Black/AA Population,Status code,Date Run,Date/Time Run
0,Illinois,2020-08-01,180476.0,7503.0,29784.0,2059.0,16.50,27.44,True,False,NaN,NaN,1824125.0,14.23,Success!,NaN,NaN
1,Louisiana,2020-07-29,116280.0,3835.0,41962.0,1884.0,44.38,50.36,False,True,94556.0,3741.0,1502916.0,32.23,Success!,NaN,NaN
2,Washington,2020-08-01,57541.0,1592.0,2134.0,51.0,5.49,3.29,False,False,38840.0,1551.0,269854.0,3.70,Success!,NaN,NaN
3,Alabama,2020-08-19,106784.0,1876.0,28664.0,769.0,41.92,42.51,False,False,68376.0,1809.0,1293186.0,26.58,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
4,Alaska,2020-08-19,2731.0,28.0,154.0,0.0,6.31,0.00,False,True,2441.0,28.0,24129.0,3.27,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
5,Arizona,2020-08-19,195557.0,4634.0,4918.0,142.0,4.50,3.55,False,False,109208.0,3997.0,305259.0,4.39,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
6,Arkansas,2020-08-19,54216.0,631.0,11909.0,163.0,24.59,25.83,False,True,48431.0,631.0,460970.0,15.41,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ


In [15]:
print(type(outer_merged))
print(type(df3sm))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [16]:
#outer_merged2 = outer_merged.merge(df3sm, how="outer")
#outer_merged2.head()
#print(outer_merged2.shape)
#outer_merged2

In [17]:
pd.concat([df1sm,df2sm,df3sm],ignore_index=True)

,Location,Date Published,Total Cases,Total Deaths,Count Cases Black/AA,Count Deaths Black/AA,Pct Cases Black/AA,Pct Deaths Black/AA,Pct Includes Unknown Race,Pct Includes Hispanic Black,Count Cases Known Race,Count Deaths Known Race,Black/AA Population,Pct Black/AA Population,Status code,Date Run,Date/Time Run
0,Illinois,2020-08-01,180476.0,7503.0,29784.0,2059,16.50,27.44,True,False,NaN,NaN,1824125.0,14.23,Success!,NaN,NaN
1,Louisiana,2020-07-29,116280.0,3835.0,41962.0,1884,44.38,50.36,False,True,94556.0,3741.0,1502916.0,32.23,Success!,NaN,NaN
2,Washington,2020-08-01,57541.0,1592.0,2134.0,51,5.49,3.29,False,False,38840.0,1551.0,269854.0,3.70,Success!,NaN,NaN
3,Alabama,2020-08-19,106784.0,1876.0,28664.0,769,41.92,42.51,False,False,68376.0,1809.0,1293186.0,26.58,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
4,Alaska,2020-08-19,2731.0,28.0,154.0,0,6.31,0.00,False,True,2441.0,28.0,24129.0,3.27,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
5,Arizona,2020-08-19,195557.0,4634.0,4918.0,142,4.50,3.55,False,False,109208.0,3997.0,305259.0,4.39,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
6,Arkansas,2020-08-19,54216.0,631.0,11909.0,163,24.59,25.83,False,True,48431.0,631.0,460970.0,15.41,Success!,2020-08-19,2020/08/19 19:34:57America/Los Angeles TZ
7,Washington,2020-07-01,33435.0,1339.0,1366.0,43.0,6.00,3.00,False,False,NaN,NaN,NaN,NaN,Success!,NaN,NaN
8,North Carolina,2020-07-01,66513.0,1373.0,10818.0,436.0,16.26,31.76,False,True,NaN,NaN,NaN,NaN,Success!,NaN,NaN


The above concatenated dataframe is as expected so lets try it for all of the CSVs located in a folder...

## Create an initial CSV of all currently available data

## Create a method which will update the latest CSV

In [18]:
%pwd
%ls

combinecsv.py
combinedData.csv
combinedData2020-08-30 21:21:47.350984.csv
combinedData2020-08-31 09:37:58.178116.csv
csv_testing_v1.ipynb
csv_testing_v2.ipynb
csv_testing_v5.ipynb
someoutputfile.csv


In [19]:
# Where's the folder of *exclusively* CSVs of interest
csvPath = '../csv-short'

#import csv files from folder
path = csvPath

allFiles = glob.glob(path + "/*.csv")
# Need to sort these
allFiles.sort()

In [20]:
allFiles

['../csv-short/covid_disparities_output_2020-05-08.csv',
 '../csv-short/covid_disparities_output_2020-05-09.csv',
 '../csv-short/covid_disparities_output_2020-05-10.csv',
 '../csv-short/covid_disparities_output_2020-05-11.csv',
 '../csv-short/covid_disparities_output_2020-05-12.csv',
 '../csv-short/covid_disparities_output_2020-05-13.csv',
 '../csv-short/covid_disparities_output_2020-05-22.csv',
 '../csv-short/covid_disparities_output_2020-06-07.csv',
 '../csv-short/covid_disparities_output_2020-06-08.csv',
 '../csv-short/covid_disparities_output_2020-06-09.csv',
 '../csv-short/covid_disparities_output_2020-06-10.csv',
 '../csv-short/covid_disparities_output_2020-06-16.csv',
 '../csv-short/covid_disparities_output_2020-06-25.csv',
 '../csv-short/covid_disparities_output_2020-06-27.csv',
 '../csv-short/covid_disparities_output_2020-06-28.csv',
 '../csv-short/covid_disparities_output_2020-06-29.csv',
 '../csv-short/covid_disparities_output_2020-06-30.csv',
 '../csv-short/covid_disparitie

In [21]:
#combine all files in the list
#dfCombined = pd.concat([pd.read_csv(f) for f in allFiles], ignore_index=True)

In [22]:
f = allFiles[0]
print(len(f))
f[-14:-4:]

52


'2020-05-08'

In [38]:
print(any(df['Student'].duplicated())) 



In [40]:
type(a)


pandas.core.series.Series

In [45]:
csvList = []
for f in allFiles:
    # Get dataframe of current file
    dfCurr = pd.read_csv(f)
    
    # Set Date Run, if not already set, from file name
    if not 'Date Run' in dfCurr.keys():
        dateCurr = f[-14:-4:]
        dfCurr['Date Run'] = dateCurr
        print('Adding Date run on ', dateCurr)
    
    # Sort current df by location
    #sort dataframe
    dfCurrSorted = dfCurr.sort_values(by='Location', ascending=True)
    
    # Drop the index
    # dfCurr.reset_index(drop=True, inplace=True)
    
    # Check for duplicate records: A location should only 
    # appear once in any given dfCurr
    if any(dfCurrSorted.duplicated(subset=['Location'])):
        print('Duplicate entry in {}'.format(f))
        print('EXITING LOOP - TODO, delete row')
        break
    
    csvList.append(dfCurrSorted)

Adding Date run on  2020-05-08
Adding Date run on  2020-05-09
Adding Date run on  2020-05-10
Adding Date run on  2020-05-11
Adding Date run on  2020-05-12
Adding Date run on  2020-05-13
Adding Date run on  2020-05-22
Adding Date run on  2020-06-07
Adding Date run on  2020-06-08
Adding Date run on  2020-06-09
Adding Date run on  2020-06-10
Adding Date run on  2020-06-16
Adding Date run on  2020-06-25
Adding Date run on  2020-06-27
Adding Date run on  2020-06-28
Adding Date run on  2020-06-29
Adding Date run on  2020-06-30
Adding Date run on  2020-07-01
Adding Date run on  2020-07-02
Adding Date run on  2020-07-03
Adding Date run on  2020-07-04
Adding Date run on  2020-07-05
Adding Date run on  2020-07-06
Adding Date run on  2020-07-07
Adding Date run on  2020-07-08
Adding Date run on  2020-07-09
Adding Date run on  2020-07-10
Adding Date run on  2020-07-11
Adding Date run on  2020-07-12
Adding Date run on  2020-07-13
Adding Date run on  2020-07-14
Adding Date run on  2020-07-15
Adding D

In [46]:
dfCombined = pd.concat(csvList, ignore_index=True)

In [47]:
#dfCombined.columns

In [48]:
dfCombined

,Unnamed: 0,Location,Date Published,Total Cases,Total Deaths,Pct Cases Black/AA,Pct Deaths Black/AA,Date Run,Status code,Count Cases Black/AA,Count Deaths Black/AA,Pct Includes Unknown Race,Pct Includes Hispanic Black,Count Cases Known Race,Count Deaths Known Race,Black/AA Population,Pct Black/AA Population,Date/Time Run,Date Run.1,Date/Time Run.1
0,Georgia,Georgia,5/8/2020,32177.0,1400.0,36.00,49.57,2020-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Massachusetts,Massachusetts,5/8/2020,75333.0,4702.0,8.18,5.25,2020-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Virginia,Virginia,5/8/2020,22342.0,812.0,17.15,22.41,2020-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Washington DC,"Washington, DC",5/8/2020,5899.0,304.0,47.35,79.28,2020-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Georgia,Georgia,5/9/2020,32504.0,1400.0,35.76,49.57,2020-05-09,Success!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,NaN,"Washington, DC",2020-08-21,13469.0,602.0,50.03,74.42,2020-08-21,Success!,6738.0,448,True,True,NaN,NaN,321317.0,46.94,2020/08/21 21:57:22America/Los Angeles TZ,NaN,NaN
2780,NaN,WestVirginia,NaN,NaN,NaN,NaN,NaN,2020-08-21,"An error occurred. ... TimeoutException('', No...",NaN,NaN,False,False,NaN,NaN,NaN,NaN,2020/08/21 21:57:22America/Los Angeles TZ,NaN,NaN
2781,NaN,Wisconsin,2020-08-21,69059.0,1068.0,14.84,20.65,2020-08-21,Success!,9274.0,217,False,True,62497.0,1051.0,368744.0,6.38,2020/08/21 21:57:22America/Los Angeles TZ,NaN,NaN
2782,NaN,Wisconsin -- Milwaukee,2020-08-21,23032.0,397.0,29.03,37.53,2020-08-21,Success!,6257.0,149,False,False,21550.0,397.0,252321.0,26.44,2020/08/21 21:57:22America/Los Angeles TZ,NaN,NaN


In [72]:
import datetime
date_object = datetime.datetime.now()
print(date_object)


2020-08-31 09:37:58.178116


In [73]:
#export to csv
dfCombined.to_csv("combinedData{}.csv".format(date_object), index=True)#, encoding='utf-8-sig')

Attempting to extract CSVs from a folder which doesn't exclusively contain *only* the CSVs we want to merge... For some reason I need to pass through the data twice to eliminate everything... ?

In [27]:
csvPath = '../csv/'

# Get all files in csv dir
allFiles = os.listdir('../csv')
#print(allFiles)

for f in allFiles:
    #print(f)
    if not ('covid_disparities_output_2020-' in f):
        allFiles.remove(f)
        #print('removed ', f)

for f in allFiles:
    #print(f)
    if not ('covid_disparities_output_2020-' in f):
        allFiles.remove(f)
        #print('removed ', f)